In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [3]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sahilsadaphal","key":"5d4e2d845079fbb88b93751cdd4a160d"}'}

In [5]:
!mkdir -p ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
!kaggle datasets download -d zalando-research/fashionmnist

Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other


In [9]:
!unzip fashionmnist.zip

Archive:  fashionmnist.zip
  inflating: fashion-mnist_test.csv  
  inflating: fashion-mnist_train.csv  
  inflating: t10k-images-idx3-ubyte  
  inflating: t10k-labels-idx1-ubyte  
  inflating: train-images-idx3-ubyte  
  inflating: train-labels-idx1-ubyte  


In [10]:
train_df=pd.read_csv('fashion-mnist_train.csv')
test_df=pd.read_csv('fashion-mnist_test.csv')

In [11]:
X_train=train_df.drop('label',axis=1).values
y_train=train_df['label'].values
X_test=test_df.drop('label',axis=1).values
y_test=test_df['label'].values

In [12]:
X_train=torch.tensor(X_train,dtype=torch.float32).to(device)
y_train=torch.tensor(y_train,dtype=torch.long).to(device)
X_test=torch.tensor(X_test,dtype=torch.float32).to(device)
y_test=torch.tensor(y_test,dtype=torch.long).to(device)

In [13]:
class CustomDataset(Dataset):
    def __init__(self, features,labels):
        self.features=features
        self.labels=labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [14]:
train_data=CustomDataset(X_train,y_train)
test_data=CustomDataset(X_test,y_test)

In [15]:
class MyNN(nn.Module):
  def __init__(self, input_dim,output_dim,hidden_layers,neurons_per_layer,dropout_rate):
      super().__init__()
      layers = []
      for i in range(hidden_layers):
          layers.append(nn.Linear(input_dim, neurons_per_layer))
          layers.append(nn.BatchNorm1d(neurons_per_layer))
          layers.append(nn.ReLU())
          layers.append(nn.Dropout(p=dropout_rate))
          input_dim = neurons_per_layer
      layers.append(nn.Linear(input_dim, output_dim))


      self.layers = nn.Sequential(*layers)

  def forward(self, x):
      return self.layers(x)

In [16]:
def objective(trial):
  # next hyperparameter values from the search space
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
  epochs = trial.suggest_int("epochs", 10, 50, step=10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
  batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
  optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

  # model init
  input_dim = 784
  output_dim = 10

  model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
  model.to(device)

   # optimizer selection
  criterion = nn.CrossEntropyLoss()

  if optimizer_name == 'Adam':
    optimizer=optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
    optimizer=optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
    optimizer=optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  # training loop

  for epoch in range(epochs):

    for batch_features, batch_labels in train_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      # forward pass
      outputs = model(batch_features)

      # calculate loss
      loss = criterion(outputs, batch_labels)

      # back pass
      optimizer.zero_grad()
      loss.backward()

      # update grads
      optimizer.step()

  # evaluation
  model.eval()
  # evaluation on test data
  total = 0
  correct = 0

  with torch.no_grad():

    for batch_features, batch_labels in test_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      outputs = model(batch_features)

      _, predicted = torch.max(outputs, 1)

      total = total + batch_labels.shape[0]

      correct = correct + (predicted == batch_labels).sum().item()

    accuracy = correct/total

  return accuracy

In [17]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 24.9 MB/s eta 0:00:00


In [18]:
import optuna

study = optuna.create_study(direction='maximize')

[I 2025-05-12 20:19:53,927] A new study created in memory with name: no-name-44e73b1f-15b7-44d8-a9b6-63d84380a4be


In [19]:
study.optimize(objective, n_trials=10)

[I 2025-05-12 20:21:12,781] Trial 0 finished with value: 0.659 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 104, 'epochs': 40, 'learning_rate': 0.045992686734270906, 'dropout_rate': 0.2, 'batch_size': 128, 'optimizer': 'RMSprop', 'weight_decay': 0.00022476873794781516}. Best is trial 0 with value: 0.659.
[I 2025-05-12 20:22:27,666] Trial 1 finished with value: 0.7486 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 56, 'epochs': 20, 'learning_rate': 3.948828264166673e-05, 'dropout_rate': 0.1, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 0.0006233920389240106}. Best is trial 1 with value: 0.7486.
[I 2025-05-12 20:30:57,574] Trial 2 finished with value: 0.87 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 56, 'epochs': 40, 'learning_rate': 7.380331544932446e-05, 'dropout_rate': 0.30000000000000004, 'batch_size': 16, 'optimizer': 'Adam', 'weight_decay': 0.00046651336098917957}. Best is trial 2 with value: 0.87.
[W 2025-05-12 20:31:25,75

KeyboardInterrupt: 